In [24]:
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.callbacks import StreamingStdOutCallbackHandler

load_dotenv()

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
    {
        "question": "For Top Gun, Could you show me three emojis that represent this movie?",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "question": "For Godfather, Could you show me three emojis that represent this movie?",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "question": "For Iron Man, Could you show me three emojis that represent this movie?",
        "answer": "🤖🦸‍♂️⚙️",
    },
]

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=80,
    return_messages=True
)

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI showing emojis to a human."),
    MessagesPlaceholder(variable_name="history"),
    few_shot_prompt,
    ("human", "{question}"),
])

def load_memory(input):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

invoke_chain("mars")
memory.load_memory_variables({})

🔴🌌🚀

{'history': [HumanMessage(content='mars'), AIMessage(content='🔴🌌🚀')]}

In [25]:
invoke_chain("king kong")
memory.load_memory_variables({})

🦍🏙️🌉

{'history': [HumanMessage(content='mars'),
  AIMessage(content='🔴🌌🚀'),
  HumanMessage(content='king kong'),
  AIMessage(content='🦍🏙️🌉')]}

In [26]:
invoke_chain("kings man")
memory.load_memory_variables({})

🤵🔫🌍

{'history': [HumanMessage(content='mars'),
  AIMessage(content='🔴🌌🚀'),
  HumanMessage(content='king kong'),
  AIMessage(content='🦍🏙️🌉'),
  HumanMessage(content='kings man'),
  AIMessage(content='🤵🔫🌍')]}